In [2]:
import tensorflow as tf
import numpy as np

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
column_names = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num',
    'marital-status', 'occupation', 'relationship', 'race', 'sex',
    'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income'
]

df = pd.read_csv(url, names=column_names, sep=', ', na_values='?', engine='python')
df.dropna(inplace=True)

y = (df['income'] == '>50K').astype(np.int32).values
X_raw = df.drop('income', axis=1)

X_processed = pd.get_dummies(X_raw, drop_first=True)
X = X_processed.values

epsilon = 1e-7
X = X.astype(np.float32)
X_mean = X.mean(axis=0)
X_std = X.std(axis=0)
X = (X - X_mean) / (X_std + epsilon)

X_train_np, X_test_np, y_train_np, y_test_np = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train = tf.constant(X_train_np, dtype=tf.float32)
X_test = tf.constant(X_test_np, dtype=tf.float32)

y_train = tf.constant(y_train_np.reshape(-1, 1), dtype=tf.float32)
y_test = tf.constant(y_test_np.reshape(-1, 1), dtype=tf.float32)

In [4]:
'''
def normalize(image):
  image = tf.cast(image, tf.float32) / 256
  image = tf.reshape(image, [-1,])
  return image
'''

'\ndef normalize(image):\n  image = tf.cast(image, tf.float32) / 256\n  image = tf.reshape(image, [-1,])\n  return image\n'

In [5]:
def create_layers(k):
  assert isinstance(k, list) and all(isinstance(i, int) for i in k) and len(k)>1, "input must be list of integers of size bigger than 1"
  return k

In [6]:
def initialize_parameters(layers_list):
  parameters = {}
  layers = len(layers_list)
  initializer = tf.keras.initializers.GlorotNormal(seed=1)
  for l in range(1, layers):
    parameters['W' + str(l)] = tf.Variable(initializer(shape=(layers_list[l], layers_list[l-1])))
    parameters['b' + str(l)] = tf.Variable(initializer(shape=(layers_list[l], 1)))
  assert len(parameters) == (layers-1) * 2
  return parameters


In [7]:
def forward_step_sig(W, A, b):
  Z = tf.math.add(tf.matmul(W, A), b)
  A = tf.keras.activations.sigmoid(Z)
  return Z, A
def forward_step_relu(W, A, b):
  Z = tf.math.add(tf.matmul(W, A), b)
  A = tf.keras.activations.relu(Z)
  return Z, A

In [8]:
def forward_propagation(X, parameters):
  L = len(parameters) // 2
  A = X
  for l in range(1,L):
    W = parameters['W' + str(l)]
    b = parameters['b' + str(l)]
    Z, A = forward_step_relu(W, A, b)
  W = parameters['W' + str(L)]
  b = parameters['b' + str(L)]
  Z, A = forward_step_sig(W, A, b)
  return Z


In [9]:
# def compute_loss(logits, labels):
#   loss = tf.keras.losses.categorical_crossentropy(tf.transpose(labels), tf.transpose(logits), from_logits = True)
#   loss = tf.reduce_sum(loss)
#   return loss
def compute_loss(logits, labels, weights_dict, lambda_l2):
  l2_term = 0.0
  for name, W in weights_dict.items():
        if len(W.shape) >= 2:
            l2_term += tf.reduce_sum(tf.square(W))

  loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
  loss = loss(labels, logits)
  loss = tf.reduce_sum(loss) + lambda_l2 * l2_term

  return loss

In [10]:
def model(X_train, Y_train, X_test, Y_test, layers, learning_rate = 0.0001, num_epochs = 1000, lambd = 0, minibatch_size = 32, print_cost = True):

  parameters = initialize_parameters(create_layers(layers))

  optimizer = tf.keras.optimizers.Adam(learning_rate)

  dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train)) # tworzy pary z obiektow tf
  test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test))

  m = dataset.cardinality().numpy() ## ilosc par
  m_test = test_dataset.cardinality().numpy()
  minibatches = dataset.batch(minibatch_size).prefetch(8)
  test_minibatches = test_dataset.batch(minibatch_size).prefetch(8)

  for epoch in range(num_epochs):
        epoch_total_loss = 0.
        for (minibatch_X, minibatch_Y) in minibatches:
            with tf.GradientTape() as tape:
                Z = forward_propagation(tf.transpose(minibatch_X), parameters)
                loss = compute_loss(Z, tf.transpose(minibatch_Y), parameters, lambd)
                trainable_variables = list(parameters.values())
                grads = tape.gradient(loss, trainable_variables)
                optimizer.apply_gradients(zip(grads, trainable_variables))
                epoch_total_loss += loss
        epoch_total_loss /= m

        if print_cost == True and epoch  % 10 == 0:
            print ("Cost after epoch %i: %f" % (epoch, epoch_total_loss*minibatch_size))
            test_cost=0
            for (minibatch_X, minibatch_Y) in test_minibatches:
                Z = forward_propagation(tf.transpose(minibatch_X), parameters)
                test_cost += compute_loss(Z, tf.transpose(minibatch_Y), parameters, lambda_l2=0)
            print(test_cost/m_test*minibatch_size)

  return parameters

In [ ]:
parameters = model(X_train, y_train, X_test, y_test, [96, 240, 144, 96, 1], learning_rate = 0.001, num_epochs = 100, lambd = 0.007, minibatch_size = 512, print_cost = True)

Cost after epoch 0: 2.771246
tf.Tensor(0.3592738, shape=(), dtype=float32)
Cost after epoch 10: 0.400179
tf.Tensor(0.33160633, shape=(), dtype=float32)
Cost after epoch 20: 0.393939
tf.Tensor(0.3305466, shape=(), dtype=float32)
Cost after epoch 30: 0.392655
tf.Tensor(0.330021, shape=(), dtype=float32)
Cost after epoch 40: 0.391994
tf.Tensor(0.3297032, shape=(), dtype=float32)
Cost after epoch 50: 0.391616
tf.Tensor(0.32932287, shape=(), dtype=float32)
Cost after epoch 60: 0.391574
tf.Tensor(0.32916576, shape=(), dtype=float32)
Cost after epoch 70: 0.391533
tf.Tensor(0.32902464, shape=(), dtype=float32)
Cost after epoch 80: 0.391064
tf.Tensor(0.32895443, shape=(), dtype=float32)
Cost after epoch 90: 0.391173
tf.Tensor(0.32892, shape=(), dtype=float32)


In [ ]:
def predict(X, parameters):
  Z_final = forward_propagation(tf.transpose(X), parameters)
  A_final = tf.nn.sigmoid(Z_final)
  predictions = tf.cast(A_final >= 0.5, dtype=tf.float32)
  return predictions

In [ ]:
pred = predict(X_test, parameters)

In [ ]:
pred2 = predict(X_train, parameters)

In [ ]:
def calculate_accuracy(predictions, labels):
    correct_predictions = tf.equal(predictions, labels)
    accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))
    return accuracy


In [ ]:
accuracy = calculate_accuracy(pred, tf.transpose(y_test))

In [ ]:
accuracy_train = calculate_accuracy(pred2, tf.transpose(y_train))

In [ ]:
print(accuracy, accuracy_train)

In [19]:
# tf.Tensor(0.8242997, shape=(), dtype=float32) tf.Tensor(0.93049854, shape=(), dtype=float32) [96, 128, 64, 64, 16, 1]
# tf.Tensor(0.8387204, shape=(), dtype=float32) tf.Tensor(0.88055867, shape=(), dtype=float32) [96, 128, 64, 64, 16, 1] with l2
# tf.Tensor(0.84816843, shape=(), dtype=float32) tf.Tensor(0.87318164, shape=(), dtype=float32)  [96, 20, 10, 50, 10, 1] l2 lambda 0.0001 lr 0.01
# tf.Tensor(0.8557931, shape=(), dtype=float32) tf.Tensor(0.86070704, shape=(), dtype=float32) [96, 20, 10, 50, 10, 1] l2 lambda 0.001
# tf.Tensor(0.85695344, shape=(), dtype=float32) tf.Tensor(0.8592565, shape=(), dtype=float32) [96, 20, 10, 50, 10, 1] l2 lambda 0.0013
# tf.Tensor(0.8541356, shape=(), dtype=float32) tf.Tensor(0.8609557, shape=(), dtype=float32) [96, 64, 16, 64, 16, 1] lr 0.01
# tf.Tensor(0.85380405, shape=(), dtype=float32) tf.Tensor(0.86157733, shape=(), dtype=float32) [96, 80, 16, 64, 16, 1]
# tf.Tensor(0.8526438, shape=(), dtype=float32) tf.Tensor(0.8769945, shape=(), dtype=float32) [96, 128, 32, 64, 64, 1] lambda 0.0025 lr 0.001
# tf.Tensor(0.8539698, shape=(), dtype=float32) tf.Tensor(0.87517095, shape=(), dtype=float32) [96, 128, 32, 64, 64, 1] lambda 0.0027 lr 0.001
# tf.Tensor(0.8567877, shape=(), dtype=float32) tf.Tensor(0.8697004, shape=(), dtype=float32) [96, 128, 32, 64, 64, 1] lambda 0.004 lr 0.001
# tf.Tensor(0.8567877, shape=(), dtype=float32) tf.Tensor(0.8651001, shape=(), dtype=float32) [96, 128, 32, 64, 128, 64, 1] labmda 0.007
# tf.Tensor(0.8567877, shape=(), dtype=float32) tf.Tensor(0.8631108, shape=(), dtype=float32) [96, 144, 32, 64, 128, 64, 1], learning_rate = 0.0006, num_epochs = 200, lambd = 0.007,
# tf.Tensor(0.8571192, shape=(), dtype=float32) tf.Tensor(0.8581375, shape=(), dtype=float32) [96, 240, 144, 96, 1], learning_rate = 0.001, num_epochs = 100, lambd = 0.007,